In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [6]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [7]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.453s

OK


In [8]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [9]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [10]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [11]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.380s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [44]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
secret = little_endian_to_int(hash256(b'bitcoinwarrior-2023'))
private_key = PrivateKey(secret=secret)
change_address = 'mi9k6g1w1e6GjhnBGhhWWpH3D9QYe4yZda'

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
prev_tx = bytes.fromhex('55b9d21121b40156ca18bd1c0b24e71fdc68f96336093a53b0b4ddcfd3095776')
prev_index = 0
tx_in = TxIn(prev_tx, prev_index)

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
amount = int(6244 * 0.6)
fee = 226
change_amount = 6244 - amount - fee
# script_pubkey = '76a9148c93ed1fc97363414322bf4c7dc61540a03611d488ac'
change_script = p2pkh_script(decode_base58(change_address))
target_script = p2pkh_script(decode_base58(target_address))

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object
tx_out_target = TxOut(amount=amount, script_pubkey=target_script)
tx_out_change = TxOut(amount=change_amount, script_pubkey=change_script)
tx = Tx(1, [tx_in], [tx_out_target, tx_out_change], 0, testnet=True)
print(tx)

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

z = tx.sig_hash(0)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
tx.tx_ins[0].script_sig = script_sig 
print(tx.serialize().hex())
# https://blockstream.info/testnet/tx/840d08458a38601599a13ddf8c33a046c6360e7fee61f03f95a265ce287945eb

tx: 1a069343a0cbde3c753b0b745c3a569b7f3cadd5c063541ad3a9dea69ad7c76f
version: 1
tx_ins:
55b9d21121b40156ca18bd1c0b24e71fdc68f96336093a53b0b4ddcfd3095776:0
tx_outs:
3746:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
2272:OP_DUP OP_HASH160 1ce576157fce2f1135aaa12020de0a11185adbf6 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
0100000001765709d3cfddb4b0533a093663f968dc1fe7240b1cbd18ca5601b42111d2b955000000006a47304402202e4b447f32c9caa10791080665496a041c11870d7f4b4658b37895c93a470f7a02201ffdc9a3d2cff485a24f33b95d6b3fb1ff0ab17918afac4fd9d28f4e668a7f89012103d826169e26694e4c3df75dde3ab0623af90771a69cb48134fac417080f9897e0ffffffff02a20e0000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ace0080000000000001976a9141ce576157fce2f1135aaa12020de0a11185adbf688ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [59]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
secret = little_endian_to_int(hash256(b'bitcoinwarrior-2023'))
private_key = PrivateKey(secret=secret)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
prev_tx = bytes.fromhex('1a9ac4b0d6129fab3db08bd4c50ec133ff7f0d3dfc581a7308044ed4c5352dca')
prev_index = 1
tx_in = TxIn(prev_tx, prev_index)

prev_tx = bytes.fromhex('840d08458a38601599a13ddf8c33a046c6360e7fee61f03f95a265ce287945eb')
prev_index = 1
tx_in_2 = TxIn(prev_tx, prev_index)

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
balance = 52272
fee = 338
amount = balance - fee
target_script = p2pkh_script(decode_base58(target_address))

# create a transaction output for the amount and address
tx_out_target = TxOut(amount=amount, script_pubkey=target_script)
tx = Tx(1, [tx_in, tx_in_2], [tx_out_target], 0, testnet=True)

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
z = tx.sig_hash(0)
z2 = tx.sig_hash(1)

der = private_key.sign(z).der()
der2 = private_key.sign(z2).der()

sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sig2 = der2 + SIGHASH_ALL.to_bytes(1, 'big')

sec = private_key.point.sec()

script_sig = Script([sig, sec])
script_sig_2 = Script([sig2, sec])

tx.tx_ins[0].script_sig = script_sig 
tx.tx_ins[1].script_sig = script_sig_2

print(tx.serialize().hex())
print(tx)
# https://blockstream.info/testnet/tx/b3f6dbdcbf2e299e75f16b0a08c9c5a854024ce64f7cdeb57a400463edcdc6d5

0100000002ca2d35c5d44e0408731a58fc3d0d7fff33c10ec5d48bb03dab9f12d6b0c49a1a010000006a47304402207ca8bdd466be36bc137c139041f4bf7b110e0c9a836f29facaf97969a107fc1a0220553e458dc23c2265bcea80d8bf27004c7986233b8f04fbe62d483c2eaa3120d2012103d826169e26694e4c3df75dde3ab0623af90771a69cb48134fac417080f9897e0ffffffffeb457928ce65a2953ff061ee7f0e36c646a0338cdf3da1991560388a45080d84010000006a473044022024015a3c055afc190fe8c37cf58798b8eb17ff3716ac4005c0a9e8d18d7fa9f202205c188305476fed95e8ee2cd09f759d77d7fc6efa124590b0daca93c7ec97fc3d012103d826169e26694e4c3df75dde3ab0623af90771a69cb48134fac417080f9897e0ffffffff01deca0000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
tx: b3f6dbdcbf2e299e75f16b0a08c9c5a854024ce64f7cdeb57a400463edcdc6d5
version: 1
tx_ins:
1a9ac4b0d6129fab3db08bd4c50ec133ff7f0d3dfc581a7308044ed4c5352dca:1
840d08458a38601599a13ddf8c33a046c6360e7fee61f03f95a265ce287945eb:1
tx_outs:
51934:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
